# Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

# Loading Data

In [5]:
season_2024_25 = pd.read_csv('data/nba_boxscores_2024-25.csv')

In [6]:
season_2024_25

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,playerSlug,position,comment,jerseyNum,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,freeThrowsMade,freeThrowsAttempted,freeThrowsPercentage,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,estimatedOffensiveRating,offensiveRating,estimatedDefensiveRating,defensiveRating,estimatedNetRating,netRating,assistPercentage,assistToTurnover,assistRatio,offensiveReboundPercentage,defensiveReboundPercentage,reboundPercentage,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE,gameMin,team_fieldGoalsMade,team_fieldGoalsAttempted,team_fieldGoalsPercentage,team_threePointersMade,team_threePointersAttempted,team_threePointersPercentage,team_freeThrowsMade,team_freeThrowsAttempted,team_freeThrowsPercentage,team_reboundsOffensive,team_reboundsDefensive,team_reboundsTotal,team_assists,team_steals,team_blocks,team_turnovers,team_foulsPersonal,team_points,team_plusMinusPoints,team_estimatedOffensiveRating,team_offensiveRating,team_estimatedDefensiveRating,team_defensiveRating,team_estimatedNetRating,team_netRating,team_assistPercentage,team_assistToTurnover,team_assistRatio,team_offensiveReboundPercentage,team_defensiveReboundPercentage,team_reboundPercentage,team_estimatedTeamTurnoverPercentage,team_turnoverRatio,team_effectiveFieldGoalPercentage,team_trueShootingPercentage,team_usagePercentage,team_estimatedUsagePercentage,team_estimatedPace,team_pace,team_pacePer40,team_possessions,team_PIE,home,won,opp_teamCity,opp_teamName,opp_teamTricode,opp_teamSlug,opp_fieldGoalsMade,opp_fieldGoalsAttempted,opp_fieldGoalsPercentage,opp_threePointersMade,opp_threePointersAttempted,opp_threePointersPercentage,opp_freeThrowsMade,opp_freeThrowsAttempted,opp_freeThrowsPercentage,opp_reboundsOffensive,opp_reboundsDefensive,opp_reboundsTotal,opp_assists,opp_steals,opp_blocks,opp_turnovers,opp_foulsPersonal,opp_points,opp_plusMinusPoints,opp_estimatedOffensiveRating,opp_offensiveRating,opp_estimatedDefensiveRating,opp_defensiveRating,opp_estimatedNetRating,opp_netRating,opp_assistPercentage,opp_assistToTurnover,opp_assistRatio,opp_offensiveReboundPercentage,opp_defensiveReboundPercentage,opp_reboundPercentage,opp_estimatedTeamTurnoverPercentage,opp_turnoverRatio,opp_effectiveFieldGoalPercentage,opp_trueShootingPercentage,opp_usagePercentage,opp_estimatedUsagePercentage,opp_estimatedPace,opp_pace,opp_pacePer40,opp_possessions,opp_PIE,gameDate
0,22400062,1610612750,Minnesota,Timberwolves,MIN,timberwolves,1630183,Jaden,McDaniels,J. McDaniels,jaden-mcdaniels,F,NaN,NaN,16:00,3,8,0.375,0,3,0.000,0,0,0.000,0,2,2,1,1,0,1,5,6,-8.0,80.0,90.0,116.4,116.7,-36.4,-26.7,0.143,1.00,10.0,0.000,0.105,0.059,10.0,0.375,0.375,0.243,0.245,95.76,90.00,75.00,30.0,-0.022,240:00,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,15,22,103,-7.0,102.1,109.6,112.2,115.8,-10.1,-6.2,0.486,1.06,13.1,0.370,0.625,0.500,15.860,17.0,0.488,0.532,1.0,0.198,99.44,94.5,78.75,94.0,0.425,True,False,Los Angeles,Lakers,LAL,lakers,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,7.0,112.2,115.8,102.1,109.6,10.1,6.2,0.524,3.14,16.3,0.375,0.630,0.500,7.143,7.4,0.468,0.519,1.0,0.200,99.44,94.5,78.75,95.0,0.575,2024-10-22
1,22400062,1610612750,Minnesota,Timberwolves,MIN,timberwolves,203944,Julius,Randle,J. Randle,julius-randle,F,NaN,NaN,34:19,5,10,0.500,1,3,0.333,5,7,0.714,3,6,9,4,0,0,2,3,16,2.0,99.5,107.5,103.1,101.4,-3.6,6.0,0.200,2.00,21.1,0.071,0.143,0.107,10.5,0.550,0.612,0.179,0.182,98.11,95.11,79.26,67.0,0.147,240:00,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,15,22,103,-7.0,102.1,109.6,112.2,115.8,-10.1,-6.2,0.486,1.06,13.1,0.370,0.625,0.500,15.860,17.0,0.488,0.532,1.0,0.198,99.44,94.5,78.75,94.0,0.425,True,False,Los Angeles,Lakers,LAL,lakers,42,9

In [ ]:
season_2023_24 = pd.read_csv('data/nba_boxscores_2023-24.csv')

In [ ]:
season_2022_23 = pd.read_csv('data/nba_boxscores_2022-23.csv')

In [ ]:
data = pd.concat([season_2024_25, season_2023_24, season_2022_23], ignore_index=True)

# Feature Engineering

## Rolling 5

In [ ]:
data['GAME_DATE'] = pd.to_datetime(data['GAME_DATE'])

data = data.sort_values(['PLAYER_NAME', 'GAME_DATE']).reset_index(drop=True)

In [ ]:
def parse_minutes_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    try:
        # Split by colon
        minutes_part, seconds_part = str(time_str).split(':')
        # Convert minute to float, in case it's like "36.000000"
        minutes = float(minutes_part)
        seconds = int(seconds_part)
        total_seconds = int(minutes * 60 + seconds)
        return total_seconds
    except:
        return None  # if formatting fails
    
data['SEC'] = data['MIN'].apply(parse_minutes_to_seconds)


In [ ]:
def rolling_5(stat):
    data[f'{stat}_rolling_5'] = (
        data.groupby('PLAYER_NAME')[stat]
            .transform(lambda x: x.shift().rolling(window=5).mean())
    )

In [ ]:
rolling_5('PTS')
rolling_5('FGM')
rolling_5('FG_PCT')
rolling_5('SEC')
rolling_5('FGA')
rolling_5('FG3M')
rolling_5('FT_PCT')
rolling_5('AST')

## Opponent

# Model

In [ ]:
player_data = data[data['PLAYER_NAME'] == 'Austin Reaves']

In [ ]:
# Select feature to predict
target = "PTS"

# Select features to train on
features = [
    "PTS_rolling_5",
    "FGM_rolling_5",
    "FG_PCT_rolling_5",
    "SEC_rolling_5",
    "FGA_rolling_5",
    "FG3M_rolling_5",
    "FT_PCT_rolling_5",
    "AST_rolling_5"
]

# Drop rows with missing data (or impute if you prefer)
player_data = player_data.dropna(subset=features + [target])

# Split into features (X) and target (y)
X = player_data[features]
y = player_data[target]

# Split train/test by time order to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)

# Train the XGBoost Regressor
model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Test RMSE: {rmse:.2f} points")

# Plot Feature Importance
plot_importance(model, max_num_features=10)
plt.title("Top 10 Feature Importances")
plt.show()